<a href="https://colab.research.google.com/github/Alex64-1149/VoxNote/blob/IA-VoxNote/VoxNote.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Adaptation des données

In [11]:

#adaptation de https://github.com/musikalkemist/pytorchforaudio

import torch
import torchaudio
import os
import linecache
from torch.utils.data import Dataset
import pandas as pd


class ReconnaissanceVocaleDataset(Dataset) :

  #Initialise les aurguments de la classe dataset:
  # 1.fichier annotation(fichier txt qui contient tous les noms des fichiers audio (wav))
  # 2.fichier audio ( contient tous les fichiers audio(wav))
  def __init__(self,FICHIER_ANNOTE,FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur, texte_transforme, nombre_caracteres): # __[...]__ = classe nécessaire a un dataset Pytorch
    self.fichierAnnotations=FICHIER_ANNOTE
    self.fichierAudio=FICHIER_AUDIO
    self.audioPathInitial = "/content/drive/MyDrive/ColabNotebooks/SiwisFrenchSpeechSynthesisDatabase/wavs/" #chemin pour se rendre au fichier pas compris dans la liste de noms de fichiers
    self.textPathInitial = "/content/drive/MyDrive/ColabNotebooks/SiwisFrenchSpeechSynthesisDatabase/text/"  #"                                                                            "
    self.processeur = processeur
    self.melSpectrogram = melSpectrogram.to(processeur) #s'assurer que tout se fasse au même endroit dans l'ordinateur : https://stackoverflow.com/questions/63061779/pytorch-when-do-i-need-to-use-todevice-on-a-model-or-tensor
    self.SAMPLE_RATE_VOULU = SAMPLE_RATE
    self.NOMBRE_ECHANTILLONS = NOMBRE_ECHANTILLONS
    self.texte_transforme = texte_transforme
    self.nombre_caracteres = nombre_caracteres

  #retourne le nombre de fichiers dans notre dataset
  def __len__(self):
    return len(self.fichierAnnotations)

  #retourne l'audio ainsi que son fichier texte associé
  def __getitem__(self, index):
    pathAudio = self.getAudioSamplePath(index)
    texte = self.getAudioSampleText(index)

    #signal informatique et sample rate de notre audio
    signal, sr = torchaudio.load(pathAudio)  #pour lire un fichier wav avec torchaudio, on obtient le format de l'onde audio ainsi que son sample rate : https://pytorch.org/audio/stable/tutorials/audio_io_tutorial.html

    #mettre le signal au même endroit que sa transformation
    signal = signal.to(self.processeur)
    #normaliser l'audio
    signal = self.resampleSiNecessaire(signal, sr)
    signal = self.combinerSiNecessaire(signal)


    #diminuer ou ajouter des échantillons "vides" si le nombre d'échantillons ne correspond à SAMPLE_RATE
    signal = self.couperSiNecessaire(signal)
    signal = self.paddingSiNecessaire(signal)


    #transformer l'audio dans le spectogram de mel
    signal = self.melSpectrogram(signal)

    #enlever la première dimension du signal qui est toujours de 1
    signal = self.modifierDimensions(signal)


    return signal, texte

  #mettre tous les fichiers audios à la même fréquence d'échantillonage
  def resampleSiNecessaire(self, signal, sr):

    if sr != self.SAMPLE_RATE_VOULU :
      resampler = torchaudio.transforms.Resample(sr, self.SAMPLE_RATE_VOULU).to(self.processeur)
      signal = resampler(signal)

    return signal

  #s'assurer que l'audio ne contient qu'une entrée et sortie (que le son ne soit pas stéréo) pour le normaliser
  def combinerSiNecessaire(self, signal):
    if signal.shape[0] > 1 :
      signal = torch.mean(signal, dim=0, keepdim= True)
    return signal

  #enlever les échantillons audios en surplus du nombre visé
  def couperSiNecessaire(self, signal) :
    #le signal est composé de 2 dimensions : [nombre de source du signal(1 dans ce cas), longueur du signal – nombre d'échantillons (on veut 22 050)]
    if signal.shape[1] > self.NOMBRE_ECHANTILLONS :
      signal = signal[:, :self.NOMBRE_ECHANTILLONS] #utilité de [:, :] : la première dimension est prise au complet et la deuxième jusqu'à l'atteinte du nombre d'échantillon (expliquer à https://youtu.be/WyJvrzVNkOc?list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&t=478)
    return signal

  def paddingSiNecessaire(self, signal) :
    #la longueur du signal = signal.shape[1] comme expliqué précédemment
    if signal.shape[1] < self.NOMBRE_ECHANTILLONS :
      paddingDuSignal = (0, self.NOMBRE_ECHANTILLONS - signal.shape[1]) #(0, nombre d'échantillons manquants)
      torch.nn.functional.pad(signal, paddingDuSignal) #ajoute le padding au signal (https://pytorch.org/docs/stable/generated/torch.nn.functional.pad.html)
    return signal


  #retourne le chemin pour avoir le bon fichier audio à un certain index du FICHIER_ANNOTE
  def getAudioSamplePath(self, index):

    index += 1 #la fonction considère la première ligne comme étant la ligne 1 (et non la ligne 0 comme habituellement)
    audioPathFin = linecache.getline(self.fichierAudio, index)
    audioPathFin = audioPathFin.strip() #pour enlever le \n : https://www.geeksforgeeks.org/python-removing-newline-character-from-string/

    return self.audioPathInitial + audioPathFin

  #retourne le fichier texte associé à l'audio d'un certain index du FICHIER_AUDIO
  def getAudioSampleText(self, index):

    index += 1
    textPathFin = linecache.getline(self.fichierAnnotations, index)
    textPathFin = textPathFin.strip()

    #encoding pour lire des textes en français trouvé sur : https://stackoverflow.com/questions/18649512/unicodedecodeerror-ascii-codec-cant-decode-byte-0xe2-in-position-13-ordinal
    fichier = open(self.textPathInitial + textPathFin,"r",  encoding="utf-8") #comment lire un fichier sur python : https://www.tutorialspoint.com/how-to-read-a-text-file-in-python
    texte = fichier.read()
    fichier.close()

    return self.convertirStringEnTensor(texte)

  def convertirStringEnTensor(self, texte): #concept pour convertir un string en tensor vient de : https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
    dimensionTensor = 64
    tensor = torch.zeros(dimensionTensor, 1, self.nombre_caracteres) #longueur maximum string * 1 * nombre caractères possibles
    for ligne, lettre in enumerate(texte):
      tensor[ligne][0][self.texte_transforme.text_to_int(lettre)] = 1 #avoir la valeur de la lettre en int dans le tableau = 1 et le reste = 0
      if ligne == dimensionTensor-1: #couper la string si ça dépasse la dimension du Tensor (comme dans couperSiNécessaire)
        break
    return tensor

  def modifierDimensions(self, tensor):
    tensorModifie = tensor[0]

    return tensorModifie



In [12]:
from google.colab import drive #nécessaire qu début de chaque session
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
#utilisé pour extraire le zip de google drive
#from google.colab import drive
#drive.mount('/content/drive')


#!unzip file.zip


# Réseau de neurone

In [14]:
#connaissances nécessaires au RNN(LSTM) trouvées à l'adresse suivante : https://ketanhdoshi.github.io/Audio-ASR/
#CNN initial vient de https://www.youtube.com/watch?v=SQ1iIKs190Q&list=PL-wATfeyAMNoirN4idjev6aRu8ISZYVWm&index=8

import torch
import torchaudio
from torch import nn
import numpy as np

class BiRNNetwork(nn.Module) : #RNN de type bidirectional LSTM

  def __init__(self, n_caracteres, rnn_dim, n_lstm_layers, dropout) :
    super().__init__()
    # 5 RNN blocks / flatten / linear /softmax
    """
    1. Normaliser les coouches à l'aide de LayerNorm pour faciliter la descente de gradient : https://www.youtube.com/watch?v=TKPowx9fb-A
    2. Créer le bi-lstm (une sous catégorie des RNN) : https://medium.com/@anishnama20/understanding-bidirectional-lstm-for-sequential-data-processing-b83d6283befc
    3. transformer en équation linéaire les données fournies : https://docs.kanaries.net/topics/Python/nn-linear
    4. normaliser les résultats à l'aide de softmax
    """
    self.n_caracteres = n_caracteres
    self.rnn_dim = rnn_dim
    self.n_lstm_layers = n_lstm_layers
    self.dropout = dropout
    #self.batchSize = batchSize #nombre d'échantillon avant de changer les paramètres : https://datascience.stackexchange.com/questions/36651/relationship-between-batch-size-and-the-number-of-neurons-in-the-input-layer


    """
    Couche de convolution du CNN qui n'a pas été utilisée pour le projet mais qui aurait pu être intéressante à utiliser en collaboration avec le LSTM pour améliorer les résultats :
    self.conv1 = nn.Sequential(
        nn.Dropout(0.1),#réduit les chances que le réseau de neurone s'adapte à une seule circonstance précise https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
        nn.Conv2d( #couche en 2 dimensions
            in_channels=inChannels, #nombre de input initial (=1 lors de l'adaptation des données)
            out_channels=outChannels, #nombre de filtre dans cette couche du réseau de neurone
            kernel_size=3, #nombre de choses analysées en même temps : https://stats.stackexchange.com/questions/296679/what-does-kernel-size-mean
            stride = 1, #déplacement du kernel : https://deepai.org/machine-learning-glossary-and-terms/stride
            padding = 2 #comme dans RSD mais pour le kernel
        ),
        nn.GELU(), #régression linéaire Gaussienne https://stackoverflow.com/questions/57532679/why-gelu-activation-function-is-used-instead-of-relu-in-bert
        nn.MaxPool2d(kernel_size=2) # reformulation des données : https://www.geeksforgeeks.org/apply-a-2d-max-pooling-in-pytorch/
    )"""

    self.layerNorm = nn.LayerNorm(rnn_dim) #réduit les valeurs des paramètres de chaque neurone du réseau pour faciliter la descente de gradient : https://www.youtube.com/watch?v=TKPowx9fb-A

    self.conv = self.sequence()

    #self.flatten = nn.Flatten() fait dim2*dim3 (cause du 65536)

    self.linear = nn.Linear(rnn_dim*2, rnn_dim) #grandeur du tensor initial, grandeur du tensor voulu https://discuss.pytorch.org/t/runtimeerror-mat1-and-mat2-shapes-cannot-be-multiplied-64x13056-and-153600x2048/101315
    self.linearFinal = nn.Linear(rnn_dim, n_caracteres)
    self.gelu = nn.GELU() #régression linéaire Gaussienne qui fonctionne mieux que la régression linéaire habituelle (RELU) https://stackoverflow.com/questions/57532679/why-gelu-activation-function-is-used-instead-of-relu-in-bert

    self.softmax = nn.Softmax(dim=1)


  def sequence(self) :
    conv = nn.Sequential(
      nn.GELU(), #régression linéaire Gaussienne https://stackoverflow.com/questions/57532679/why-gelu-activation-function-is-used-instead-of-relu-in-bert
      nn.LSTM(self.rnn_dim, #nombre de input initial
              num_layers=self.n_lstm_layers, #nombre de couches passées à travers avant de retourner une valeur : https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm
              hidden_size= self.rnn_dim, #nombre de composantes des vecteurs représentant les valeurs : https://stackoverflow.com/questions/75648914/trying-to-understand-lstm-parameter-hidden-size-in-pytorch#:~:text=The%20hidden_size%20is%20a%20hyper,hyper%2Dparameter%20(%20num_layers%20).
              dropout=self.dropout, #réduit les chances que le réseau de neurone s'adapte à une seule circonstance précise https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
              bidirectional=True), #passe à travers les données dans les deux sens
    )
    return conv

  def forward(self, input_data): #traitement des données dans le réseau de neurone

    """LSTM DE 5 de profondeur"""
    x = self.layerNorm(input_data) #normaliser les données
    x, hidden_state  = self.conv(x) #LSTM de 5 de profondeurs | retourne les variables : https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/#:~:text=The%20output%20of%20nn.,which%20is%20not%20used%20here.
    x = self.linear(x) #modifier les dimensions de 1024 à 512


    x = self.layerNorm(x)
    x, _ = self.conv(x) #profondeur 2
    x = self.linear(x)


    x = self.layerNorm(x)
    x, _ = self.conv(x) #profondeur 3
    x = self.linear(x)


    x = self.layerNorm(x)
    x, _ = self.conv(x) #profondeur 4
    x = self.linear(x)


    x = self.layerNorm(x)
    x, _ = self.conv(x) #profondeur 5
    x = self.linear(x)

    x = self.gelu(x)
    logits = self.linearFinal(x) #logits signifie la probabilité (avant d'être normalisé) associée à certaines réponses : https://www.linkedin.com/posts/mwitiderrick_what-are-logits-in-deep-learning-logits-activity-7084819307959902209-UUGe

    predictions = self.softmax(logits) #normaliser les probabilités
    return predictions



#algorithme pour le mapping prit et adapté depuis : https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
char_map_str = """
 ä 0
 ’ 1
 a 2
 b 3
 c 4
 d 5
 e 6
 f 7
 g 8
 h 9
 i 10
 j 11
 k 12
 l 13
 m 14
 n 15
 o 16
 p 17
 q 18
 r 19
 s 20
 t 21
 u 22
 v 23
 w 24
 x 25
 y 26
 z 27
 - 28
 à 29
 â 30
 é 31
 è 32
 ê 33
 ë 34
 î 35
 ï 36
 ô 37
 ù 38
 û 39
 ü 40
 ÿ 41
 ç 42
 æ 43
 œ 44
 , 45
 . 46
 … 47
 ( 48
 ) 49
 « 50
 » 51
 ! 52
 ? 53
 0 54
 1 55
 2 56
 3 57
 4 58
 5 59
 6 60
 7 61
 8 62
 9 63
 """
#associer des caractères à des valeurs numériques
class TextTransform:
  """Maps characters vers integers et vice versa"""
  def __init__(self, char_map_str):
      self.char_map_str = char_map_str
      self.char_map = {}
      self.index_map = {}
      for line in char_map_str.strip().split('\n'):
          ch, index = line.split()
          self.char_map[ch] = int(index)
          self.index_map[int(index)] = ch
      self.index_map[0] = ' '

      charList = []
      for key in self.char_map_str:
        charList.append(key)

      self.charTuple = tuple(charList)

  def text_to_int(self, text):
      """ Converti le texte en séquence de int à l'aide d'une map de texte à int """
      text = text.lower()
      int_sequence = []
      for c in text:
          if c == ' ' or not(c in self.char_map_str):
              ch = self.char_map['ä']#on utilise ä comme équivalent de ' '
          else:
              ch = self.char_map[c]
          int_sequence.append(ch)
      return int_sequence

  def int_to_text(self, labels):
      """ Converti une séquence de int en séquence de texte à l'aide d'une map de int à texte """
      string = []
      for i in labels:
        string.append(self.charTuple[int(i)])
      return ''.join(string).replace('ä', ' ')




# Entraîner et Tester

In [15]:
import torch
import torchaudio
from torch.utils.data import DataLoader
import torch.nn.functional as F

#paramètres essentiels lors de machine learning
hyper_parameters = { #paramètres qui décident comment se déroulera l'entrainement : https://aws.amazon.com/fr/what-is/hyperparameter-tuning/
        "n_lstm_layers": 2, #nombre de couches de lstm
        "rnn_dim": 512, #inChannels
        "n_caracteres": 65, #dimensionOutput
        "dropout": 0.1, ##réduit les chances que le réseau de neurone s'adapte à une seule circonstance précise https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html
        "learning_rate": 0.0001, #vitesse d'apprentissage
        "batch_size": 20, #nombre d'éléments par endoit qu'on entraine
        "epochs": 10 #nombre de fois qu'on entraine le réseau au complet
    }

"""entrainer le RNN à l'aide du CTC Algorithm, un algorithme qui sert à déterminer où sont placer les lettres dans un fichier audio : https://ketanhdoshi.github.io/Audio-ASR/"""


def entrainerEpoque(modele, chargeurDonnees, criterion, optimizer, scheduler, epoch, processeur):

  modele.train() #spécifie que les valeurs du modèles vont être modifiées (pour annuler modele.eval())
  print(f"Entrainement de l'époque {epoch}")

  for audios, textesAssocies in chargeurDonnees :

    #s'assurer que l'audio et le texte soit au même endroit que le modèle
    audios = audios.to(processeur)
    textesAssocies = textesAssocies.to(processeur)


    #réinitialise les paramètres pour permettre au réseau de neurone de ne pas s'encombrer de paramètres précédents inutiles : https://medium.com/@lazyprogrammerofficial/in-pytorch-why-do-we-need-to-call-optimizer-zero-grad-8e19fdc1ad2f
    optimizer.zero_grad()

    #faire une prédiction et déterminer de comment il faut modifier notre réseau de neurone, à l'aide du CTCLoss, pour améliorer la prédiction
    prediction = modele(audios)

    #enlever les dimensions de 1 du tensor
    textesAssocies = textesAssocies.squeeze()

    prediction = prediction.transpose(0, 1) #interchange les dimensions pour obtenir (input, batch, classes) https://pytorch.org/docs/stable/generated/torch.transpose.html

    cible = textesAssocies.transpose(0,1).transpose(0,2)#pour obtenir la forme (batch, longueur maximale) avec la prochaine ligne de code
    cible = cible[0] #voir ligne précédente


    #avoir un input_length = input_length_cible = batch size

    input_lengths = []
    for input_length in audios :
      input_lengths.append(input_length.shape[0]) #shape = (64, 512)



    input_length_cible = []
    for texte in textesAssocies :
      input_length_cible.append(64) #64 comme déterminé dans le dataset

    input_lengths = torch.tensor(tuple(input_lengths)) #transformer le tuple en tensor pour l'utiliser avec le criterion
    input_length_cible = torch.tensor(tuple(input_length_cible))

    divergence = criterion(prediction, cible, input_lengths, input_length_cible) # variables dans le CTCloss : https://pytorch.org/docs/stable/generated/torch.nn.CTCLoss.html

    #update les valeurs du réseau de neurone avec une backpropagation
    divergence.backward() # modifie le poids de chaque paramètres selon la divergence calculée (est-ce qu'il faut augmenter ou diminuer la valeur de ce neurone) : https://en.wikipedia.org/wiki/Backpropagation

    #passage au prochain paramètre de l'optimizer et du scheduler
    optimizer.step() #descente de gradient : https://stackoverflow.com/questions/53975717/pytorch-connection-between-loss-backward-and-optimizer-step
    scheduler.step() #modifie le learning rate : https://discuss.pytorch.org/t/what-does-scheduler-step-do/47764

def tester(modele, chargeurDonnees, criterion, epoch, processeur, texte_transforme, caractere_vide) :

  modele.eval() # rend fixe certains paramètres qui sont changés durant l'entraînement : https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
  divergence_test = 0
  erreurLettre = 0
  nombreLettre = 0

  with torch.no_grad() : #même chose que model.eval() sur d'autres paramètres
    for audios, textesAssocies in chargeurDonnees :
      #code comme le train en enlevant toutes les commandes pour modifier les paramètres du réseau de neurone
      audios = audios.to(processeur)
      textesAssocies = textesAssocies.to(processeur)

      texteAssocieInitial = textesAssocies.squeeze().tolist()

      prediction = modele(audios)
      textesAssocies = textesAssocies.squeeze()

      prediction = prediction.transpose(0, 1) #interchange les dimensions pour obtenir (input, batch, classes) https://pytorch.org/docs/stable/generated/torch.transpose.html
      cible = textesAssocies.transpose(0,1).transpose(0,2)#pour obtenir la forme (batch, longueur maximale)
      cible = cible[0]

      #avoir un input_length = input_length_cible = batch size

      input_lengths = []
      for input_length in audios :
        input_lengths.append(input_length.shape[0]) #shape = (64, 512)


      input_length_cible = []
      for texte in textesAssocies :
        input_length_cible.append(64)

      input_lengths = torch.tensor(tuple(input_lengths))
      input_length_cible = torch.tensor(tuple(input_length_cible))

      divergence = criterion(prediction, cible, input_lengths, input_length_cible)
      divergence_test += divergence

      #transformer les valeurs du spectogram en valeurs numériques et ensuite en texte
      predictions_decode, valeurs_decode = GreedyDecoder(prediction, texteAssocieInitial, texte_transforme, caractere_vide) #retourne la prédiction et sa valeur associée
      nombreLettre += len(valeurs_decode) #nombre de lettres utilisées
      for i in range(len(predictions_decode)) :
        if i < len(valeurs_decode) :
          if predictions_decode[i] != valeurs_decode[i] :
            erreurLettre += 1
        else : #plus de lettres prédites que prévu
          erreurLettre += 1

    pourcentErreur = erreurLettre/nombreLettre*100
    print(f"Le pourcentage d'erreur à l'époque {epoch} = {pourcentErreur}% pour les lettres et la divergence(loss) = {divergence_test}")

"""
fonction courante dans les speech to text qui compare le résultat attendu avec le résultat obtenu
le format de celui-ci est inspiré par : https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
"""
def GreedyDecoder(prediction, texteAssocie, texte_transforme, caractere_vide) :

  decode = []
  #prediction : contient 20 batch -> prediction[x] = tous les tensors représentant les caractères -> prediction[x][y] = probabilité de chaque caractère
  for valeurs in enumerate(prediction) : #valeurs correspond au caracère à chaque endroit possible (prediction contient : [tensor, device utilisé])
    for probabilite in enumerate(valeurs[1]) : #index correspond au caractère à un endroit précis
      index = torch.argmax(probabilite[1])
      if index != caractere_vide:
        decode.append(index)


  attendu = []
  for texte in enumerate(texteAssocie) :
    for index in enumerate(texte[1]) : #texte = (#de la ligne, [[0.0,...,0.0]]) on veut donc itérer dans la double liste
      char = index[1].index(max(index[1])) #pour obtenir la position de la valeur 1.0
      if char != caractere_vide:
        attendu.append(char)


  attendu = texte_transforme.int_to_text(attendu) #valeurs du texte
  decode = texte_transforme.int_to_text(decode) #valeurs du LSTM

  return decode, attendu


FICHIER_ANNOTE = "/content/drive/MyDrive/ColabNotebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_text.list"
FICHIER_AUDIO = "/content/drive/MyDrive/ColabNotebooks/SiwisFrenchSpeechSynthesisDatabase/lists/all_wavs.list"


#nombre d'échantillons par secondes dans notre audio
SAMPLE_RATE = 44*512-1 #n_fft * hop_length - 1 à cause de la formule n_fft : https://pytorch.org/audio/main/generated/torchaudio.transforms.MelSpectrogram.html
NOMBRE_ECHANTILLONS = 44*512-1

if torch.cuda.is_available(): #détermine ce qui exécute le programme (gpu préférable pour AI audio)
    processeur = "cuda"
else:
    processeur = "cpu"

#le Spectogram de Mel est une échelle logarithmique utilisée pour mieux représenter les différences qu'un humain entend dans un fichier audio ce qui aide à l'analyse sonore
melSpectrogram = torchaudio.transforms.MelSpectrogram(
    SAMPLE_RATE,
    n_fft=512, #longueur physique du signal optimale pour la reconnaissance vocale selon : https://librosa.org/doc/main/generated/librosa.stft.html
    hop_length=44, #nombre d'échantillon audio adjacents analysés par la transformée de fourier : https://librosa.org/doc/main/generated/librosa.stft.html
    n_mels=128 #nombre de séparations d'une seule fréquence optimale pour la reconnaisance vocale selon:https://stackoverflow.com/questions/62623975/why-128-mel-bands-are-used-in-mel-spectrograms
)

texte_transforme = TextTransform(char_map_str)

#initialisation du Dataset
rvd = ReconnaissanceVocaleDataset(FICHIER_ANNOTE, FICHIER_AUDIO, melSpectrogram, SAMPLE_RATE, NOMBRE_ECHANTILLONS, processeur, texte_transforme,hyper_parameters["n_caracteres"])




#https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
train_data_loader = DataLoader(rvd,
                              hyper_parameters["batch_size"],
                              shuffle=True, #https://discuss.pytorch.org/t/how-does-shuffle-in-data-loader-work/49756/7
                              )
test_data_loader = DataLoader(rvd, #préférablement pas tt le dataset(rvd)
                              hyper_parameters["batch_size"],
                              shuffle=False,
                              )

#initialisation du réseau de neurones
rnn = BiRNNetwork(hyper_parameters["n_caracteres"], hyper_parameters["rnn_dim"], hyper_parameters["n_lstm_layers"], hyper_parameters["dropout"])
rnn = rnn.to(processeur) #to(processeur) s'assure que tout s'entraine au même endroit (sur le cuda dans ce cas ci)

#le choix du optimizer et du scheduler a été effectué selon l'article : https://www.assemblyai.com/blog/end-to-end-speech-recognition-pytorch/
"""
On utilise le CTCLoss function dans le speech to text pour aligner les endroits où il est prédit qu'il y ait des lettres avec les bons neurones (Doshi, 2021)
blank permet de ne pas tenir compte des endroits où on prédit qu'il n'y aura pas de caractères : https://distill.pub/2017/ctc/?undefined=&ref=assemblyai.com
"""
criterion = nn.CTCLoss(blank = hyper_parameters["n_caracteres"]-1).to(processeur) #calcul les probabilités selon une fonction prédéfinie : https://nn.readthedocs.io/en/rtd/criterion/index.html

optimizer = torch.optim.AdamW(rnn.parameters(), hyper_parameters["learning_rate"]) #change les paramètres du modèle pour améliorer la performance : https://towardsdatascience.com/optimizers-for-training-neural-network-59450d71caf6

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hyper_parameters["learning_rate"],
                                                epochs=hyper_parameters["epochs"], # paramètres nécessaires au scheduler :https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.OneCycleLR.html
                                                steps_per_epoch=len(rvd) # nombre de neurones qu'on entraîne par epoch : https://community.deeplearning.ai/t/request-for-explanation-on-steps-per-epoch-parameter/501777
                                                ) #modifie le learning rate pour améliorer la performance : https://towardsdatascience.com/learning-rate-scheduler-d8a55747dd90


for epoch in range(hyper_parameters["epochs"]):
  entrainerEpoque(rnn, train_data_loader, criterion, optimizer, scheduler, epoch, processeur)
  if epoch % 2 == 0 :
    tester(rnn, test_data_loader, criterion, epoch, processeur, texte_transforme, hyper_parameters["n_caracteres"])


torch.save(rnn.state_dict(), "feedforwardnet.pth") #sauvegarder le modele : https://pytorch.org/tutorials/beginner/basics/saveloadrun_tutorial.html
print("rnn entraîné sauvegardé sur feedforwardnet.pth")

Entrainement de l'époque 0


/usr/local/lib/python3.10/dist-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (257) may be set too low.
  warnings.warn(


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
        0.0078, 0.0078]))
index tensor(38)
probabilite (1, tensor([0.0078, 0.0079, 0.0078, 0.0078, 0.0078, 0.0079, 0.0079, 0.0078, 0.0078,
        0.0079, 0.0078, 0.0078, 0.0079, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078,
        0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078,
        0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0079, 0.0078,
        0.0078, 0.0078, 0.0079, 0.0078, 0.0078, 0.0078, 0.0078, 0.0079, 0.0078,
        0.0078, 0.0078, 0.0078, 0.0079, 0.0078, 0.0078, 0.0078, 0.0078, 0.0079,
        0.0078, 0.0078, 0.0079, 0.0078, 0.0078, 0.0079, 0.0078, 0.0078, 0.0078,
        0.0078, 0.0078]))
index tensor(38)
probabilite (2, tensor([0.0078, 0.0079, 0.0078, 0.0078, 0.0078, 0.0079, 0.0079, 0.0078, 0.0078,
        0.0079, 0.0078, 0.0078, 0.0079, 0.0078, 0.0078, 0.0078, 0.0078, 0.0078,
        0.0078, 0.0078, 0.0078, 0.0078, 0.0078, 0.0079, 0.0078, 0.0078, 0.0078,
    

# Inférence

In [16]:
#"""Informations pour créer la classe inférence trouvé sur :https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html """
#
#def inference() :
#  with torch.no_grad():
#    inference = rnn(audio)
#    texteDeduit = GreedyDecoderInference(inference, texte_transforme, caractere_vide)
#
#def GreedyDecoderInference(inference, texte_transforme, caractere_vide) :
#